This notebook is intended to be downloaded and run locally, or run in cloud environments with persistent environments, like Sagemaker Studio Lab:

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/InsightSoftwareConsortium/itkwidgets/HEAD?labpath=examples%2FEnvironmentCheck.ipynb)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github.com/InsightSoftwareConsortium/itkwidgets/blob/main/examples/EnvironmentCheck.ipynb)

# Environment Check

#### This notebook is designed to check the environment that you are running in to make sure that all required dependencies and extensions are correctly installed. Simply select Run All Cells and let everything complete before running the other notebooks in this repository.

In [ ]:
import os, sys, re
import importlib.util
import importlib.metadata

#### Define the function to do the checking and install any missing dependencies

In [ ]:
def _get_version(pkg):
    try:
        return importlib.metadata.version(pkg)
    except:
        pass
    try:
        return sys.modules[pkg].__version__
    except:
        return ''

def check_for_package(req, version=None):
    vals = re.split(r"(\[.*\]|==|>=)", req)
    pkg = vals[0]
    if len(vals) > 1:
        version = vals[-1]
    if importlib.util.find_spec(pkg) is None or (version and _get_version(pkg) != version):
        print(f"{pkg} not found, installing {req} now...")
        !{sys.executable} -m pip install -q {req}
    print(f"{pkg} version {_get_version(pkg)} installed.")
    print("-----")

#### List of notebook requirements

In [ ]:
reqs = [
    "itkwidgets[all]==1.0a23",
    "imjoy-elfinder",
    "imjoy-jupyter-extension",
    "imjoy-jupyterlab-extension",
    "itk==5.3.0",
    "itk-io==5.3.0",
    "itk-core",
    "monai-weekly[nibabel, matplotlib, tqdm]",
    "imageio",
    "pyvista",
    "dask[diagnostics]",
    "toolz",
    "scikit-image",
    "pooch",
    "matplotlib",
    "tqdm",
    "vtk",
    "netCDF4",
    "xarray",
    "zarr",
    "fsspec[http]",
]

#### Upgrade pip, just in case.

In [ ]:
!{sys.executable} -m pip install --upgrade -q pip

#### Make sure that the package is installed and that it is the correct version.

In [ ]:
for req in reqs:
    check_for_package(req)

In [ ]:
if os.environ.get('CONDA_DEFAULT_ENV', None):
    !conda install --yes -q --prefix {sys.prefix} -c conda-forge pyimagej
else:
    raise RuntimeError("No conda environment is activated, currently unable to install pyimagej. Please activate a conda environment and re-run this cell.")

#### Special case specific to running in AWS StudioLab

In [ ]:
if "studio-lab-user" in os.getcwd():
    # Make sure that the imjoy extension is installed in the Jupyter environment
    # and not just the kernel environment since they may not be the same
    !conda env update -n studiolab -f ../environment.yml
    !conda install --yes -q --prefix {sys.prefix} -c conda-forge opencv nodejs

#### Make sure that the required extension(s) are loaded.

In [ ]:
%%javascript
if (typeof window.loadImJoyRPC === "undefined") {
  location.reload()
}